# [Advent of Code 2020 Day 10](https://adventofcode.com/2020/day/10)

This looks nasty.

## Initial setup

In [1]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Part 1 - Cheesing It
This question has a lot of parallels with CSPs you'll find in AI/CI courses. I was originally going to do a backtracker, but then I made a realization in the form of transform-and-conquer...

### The Joltage Diff Counter
This will just take an array of nums and multiply the number of 1-diffs by the number of 3-diffs. If there exists a diff outside the 1-3 range it throws.

In [2]:
def get_product_of_1_and_3_jolt_differences(nums: list[int]) -> int:
    diff_1 = 0
    diff_3 = 0

    for i in range(1, len(nums)):
        if nums[i] - nums[i - 1] == 1:
            diff_1 += 1
        elif nums[i] - nums[i - 1] == 3:
            diff_3 += 1
        elif nums[i] - nums[i - 1] == 2:
            continue
        else:
            raise Exception(f"For array {nums} the diff for {nums[i]=} and {nums[i - 1]=} doesn't satisfy the 1-3 rule")

    return diff_1 * diff_3

In [3]:
%%ipytest
def test_get_product_of_1_and_3_jolt_differences():
    assert get_product_of_1_and_3_jolt_differences([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]) == 35

.                                                                                            [100%]
1 passed in 0.01s


### Transform and Conquer
We want every element. The wording of this question implies it always exists. Therefore, we can just sort the input and run the difference function on that.

In [4]:
def get_multiplied_diffs(nums: list[int]) -> int:
    """
    Takes a list of numbers and orders them so the maximum absolute difference is 3, then returns the number of 1-diffs multiplied by the number of 3-diffs.
    :param nums: numbers to examine
    :return: the product of 1-diffs and 3-diffs
    """
    return get_product_of_1_and_3_jolt_differences([0] + sorted(nums + [max(nums) + 3]))

In [5]:
%%ipytest
def test_get_multiplied_diffs():
    assert get_multiplied_diffs([16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]) == 35

.                                                                                            [100%]
1 passed in 0.01s


## Part 2 - Cheesing It Again
Looking at the example on the question page,
```
(0), 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 6, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 6, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 6, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 7, 10, 12, 15, 16, 19, (22)
```
we notice that sorting the input gives it an interesting property: it represents the longest possible answer. Every possible answer after it involves some "contraction" of some elements. For example, if you look at the first six elements of the top-most row following the default `(0)`, `[1, 4, 5, 6, 7, 10]`, we see that the shortest possible form of it occurs as `[1, 4, 7, 10]` at the bottom, as the `[5, 6]` are contracted and disappear.

Examining the canonical sorted result, we see (notated below the array element), the number of elements to the right of a given element it can contract plus 1; this roughly translates to how many times a number can "choose" its destiny:
```
[ 1,  4,  5,  6,  7, 10, 11, 12, 15, 16, 19]
[ 2   3   3   2   2   3   2   2   2   2   1]
```
Using the "rule of product" in combinatorics, my first idea was to multiply everything together, as if there are 3 ways to choose a burger, and 4 ways to choose a drink, there are 12 ways in total to choose a burger, and drink! However, I think this may result in erroneous counts, because if an element is contracted, I shouldn't be allowed to multiply its number together (as it wouldn't be present in the candidate array).

But that's for later. I still need to make the counter function first. Let's do that.

### Contractions

In [6]:
def get_contractions(nums: list[int]) -> list[int]:
    """
    For each number, return an array representing, at each index, how many times the number can "choose its destiny".
    Assumes array is sorted and only contains distinct elements.
    :param nums:
    :return:
    """

    # Guardrails; can delete later
    assert nums == sorted(nums)
    assert len(set(nums)) == len(nums)

    contractions = [0] * len(nums)

    for i in range(len(nums)):
        if i + 1 < len(nums) and nums[i + 1] - nums[i] <= 3:
            contractions[i] += 1
        if i + 2 < len(nums) and nums[i + 2] - nums[i] <= 3:
            contractions[i] += 1

    return contractions

In [7]:
%%ipytest
def test_get_contractions():
    assert get_contractions([1, 2, 3]) == [2, 1, 0]
    assert get_contractions([1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19]) == [1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 0]
    assert get_contractions([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]) == [1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0]

.                                                                                            [100%]
1 passed in 0.01s


### Rule of Product?
Let's see what happens if we multiply everything together except the 0 at the end...

In [8]:
def get_product_of_nonzero_elements(nums: list[int]) -> int:
    assert nums and nums[-1] == 0 and nums.count(0) == 1
    result = 1
    for num in nums:
        if num == 0:
            break
        result *= num
    return result

In [9]:
%%ipytest
def test_get_product_of_nonzero_elements():
    assert get_product_of_nonzero_elements([1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 0]) == 8
    assert get_product_of_nonzero_elements([1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0]) == 8

.                                                                                            [100%]
1 passed in 0.01s


Unfortunately, this only passes the smaller `example1` but not `example2`. For the latter, it returned 32768 instead of 19208. I'm erroneously counting elements more than once.

### Top-Down?
I'm going to try the following recurrence relation:
$f(n) = \prod_{i=1}^{cons} f(n+i) \text{ where cons is the number of contractions}$

In [10]:
def count_adapter_configurations(nums: list[int], input_contractions: list[int] = None) -> int:

    # Guardrails; can delete later
    assert sorted(nums) == nums
    assert len(set(nums)) == len(nums)

    contractions = get_contractions(nums) if input_contractions is None else input_contractions

    print(f"contractions for {nums=}:")
    print(f"{contractions}")

    assert contractions and contractions[-1] == 0
    contractions.pop()

    def f(idx: int) -> int:
        if idx >= len(contractions):
            return 1
        answer = contractions[idx]
        for i in range(1, contractions[idx] + 1):
            answer *= f(idx + i)
        return answer

    return f(0)

In [11]:
%%ipytest
def test_count_adapter_configurations_basic():
    assert count_adapter_configurations([0, 3, 6]) == 1
    assert count_adapter_configurations([0, 3, 4, 6]) == 2
    assert count_adapter_configurations([3, 4, 5, 6]) == 4
    assert count_adapter_configurations([0, 3, 4, 5, 6, 9]) == 4
    assert count_adapter_configurations([0, 2, 3, 4, 5, 6, 9]) == 24

def test_count_adapter_configurations_advanced():
    assert count_adapter_configurations([], input_contractions=[1, 2, 1, 1, 0]) == 2
    assert count_adapter_configurations([], input_contractions=[2, 2, 1, 1, 2, 1, 0]) == 8
    assert count_adapter_configurations([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]) == 8

FF                                                                                           [100%]
============================================ FAILURES =============================================
_____________________________ test_count_adapter_configurations_basic _____________________________

    def test_count_adapter_configurations_basic():
        assert count_adapter_configurations([0, 3, 6]) == 1
        assert count_adapter_configurations([0, 3, 4, 6]) == 2
        assert count_adapter_configurations([3, 4, 5, 6]) == 4
        assert count_adapter_configurations([0, 3, 4, 5, 6, 9]) == 4
>       assert count_adapter_configurations([0, 2, 3, 4, 5, 6, 9]) == 24
E       assert 128 == 24
E        +  where 128 = count_adapter_configurations([0, 2, 3, 4, 5, 6, ...])

C:\Users\Jason\AppData\Local\Temp\ipykernel_37864\1643940141.py:6: AssertionError
-------------------------------------- Captured stdout call ---------------------------------------
contractions for nums=[0, 3, 6]:
[

Agh this isn't working! I think I need to use rule of sum instead of rule of product.

In [12]:
def count_configurations_top_down(nums: list[int], idx: int, dp: dict[int, int]) -> int:

    if (hit := dp.get(idx)) is not None:
        return hit

    if idx >= len(nums) - 1:
        return 1

    answer = 0

    if idx + 3 < len(nums) and nums[idx + 3] - nums[idx] <= 3:
        answer += count_configurations_top_down(nums, idx + 3, dp)

    if idx + 2 < len(nums) and nums[idx + 2] - nums[idx] <= 3:
        answer += count_configurations_top_down(nums, idx + 2, dp)

    if idx + 1 < len(nums) and nums[idx + 1] - nums[idx] <= 3:
        answer += count_configurations_top_down(nums, idx + 1, dp)

    dp[idx] = answer
    return answer

In [13]:
%%ipytest
def test_count_configurations_top_down():
    assert count_configurations_top_down([0, 3, 6], 0, {}) == 1
    assert count_configurations_top_down([0, 3, 4, 6], 0, {}) == 2
    assert count_configurations_top_down([3, 4, 5, 6], 0, {}) == 4
    assert count_configurations_top_down([0, 3, 4, 5, 6, 9], 0, {}) == 4
    assert count_configurations_top_down([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22], 0, {}) == 8

.                                                                                            [100%]
1 passed in 0.01s


No shot, was that it? Can we go bottom-up?

In [14]:
def count_configurations_bottom_up(nums: list[int]) -> int:

    arr = [0] * len(nums)

    arr[-1] = 1

    for i in reversed(range(len(nums) - 1)):
        if i + 3 < len(nums) and nums[i + 3] - nums[i] <= 3:
            arr[i] += arr[i + 3]
        if i + 2 < len(nums) and nums[i + 2] - nums[i] <= 3:
            arr[i] += arr[i + 2]
        if i + 1 < len(nums) and nums[i + 1] - nums[i] <= 3:
            arr[i] += arr[i + 1]

    return arr[0]

In [15]:
%%ipytest
def test_count_configurations_bottom_up():
    assert count_configurations_bottom_up([0, 3, 6]) == 1
    assert count_configurations_bottom_up([0, 3, 4, 6]) == 2
    assert count_configurations_bottom_up([3, 4, 5, 6]) == 4
    assert count_configurations_bottom_up([0, 3, 4, 5, 6, 9]) == 4
    assert count_configurations_bottom_up([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]) == 8

.                                                                                            [100%]
1 passed in 0.01s


## Main Solver

In [16]:
def solve(prob, filename):
    lines = []
    gen = yield_line(filename)

    for line in gen:
        lines.append(line)

    nums = list(map(int, lines))

    if prob == 1:
        return get_multiplied_diffs(nums)
    elif prob == 2:
        return count_configurations_bottom_up([0] + sorted(nums) + [max(nums) + 3])
    else:
        print("Invalid problem code")
        exit()

In [17]:
%%ipytest
def test_solve():
    assert solve(1, "example1") == 35
    assert solve(1, "example2") == 220
    assert solve(1, "input") == 2380
    assert solve(2, "example1") == 8
    assert solve(2, "example2") == 19208
    assert solve(2, "input") == 48358655787008

.                                                                                            [100%]
1 passed in 0.01s


No fucking way. It actually worked.

Instead of multiplying every method count together, I had to add them. Because the ways were dependent on each other and incapable of being done simultaneously.